In [29]:
!pip install cbastats
!pip install pymysql
!pip install openpyxl

     |████████████████████████████████| 172 kB 1.6 MB/s eta 0:00:01
  Created wheel for openpyxl: filename=openpyxl-3.0.3-py2.py3-none-any.whl size=241262 sha256=98f3fe8ec3e1ca28adfdb01375d8df94f210412c30cf626f926a83eb319e7a81
  Stored in directory: /root/.cache/pip/wheels/5b/c0/e7/e16d3916984addf40bd021fa98d1624cb24a9adc48da4e62b5
  Created wheel for et-xmlfile: filename=et_xmlfile-1.0.1-py3-none-any.whl size=8915 sha256=7f9a23a9d8488a3ca613b84f4ae26feb87f1f032be0cb3bfab0b6a30886e4e07
  Stored in directory: /root/.cache/pip/wheels/e2/bd/55/048b4fd505716c4c298f42ee02dffd9496bb6d212b266c7f31
Successfully built openpyxl et-xmlfile


In [1]:
from cbastats.Team import *
from cbastats.Player import *
from sqlalchemy import create_engine
import datetime

In [2]:
print('最近一次运行时间：',datetime.datetime.now())

最近一次运行时间： 2020-07-30 01:16:45.147213


# 载入数据

In [8]:
user_name = 'guest'
passcode = 'Guest123456'
endpoint = 'cbashuju.ctkaehd5rxxe.us-east-1.rds.amazonaws.com'
database = 'CBA_Data'
engine = create_engine(f'mysql+pymysql://{user_name}:{passcode}@{endpoint}/{database}')

connection= engine.connect()

In [9]:
df = pd.read_sql("select * from CBA_Data.PlayerStatsPerGame", connection);

In [10]:
connection.close()

In [17]:
df.head().transpose()

,0,1,2,3,4
球员,布鲁克斯,任骏飞,易建联,赵睿,胡明轩
号码,6,20,9,10,3
出场时间,36,30,35,31,13
首发,1,1,1,1,1
进攻篮板,0,1,1,2,0
防守篮板,3,1,11,5,1
助攻,9,1,2,4,0
犯规,3,3,2,3,1
抢断,2,3,1,2,0
失误,1,0,1,0,1


In [143]:
whole_season = df
later_season = df.loc[df['Game_ID']>'2019202000300'].copy()

# 所有球队高阶数据统计

In [144]:
# 整个赛季数据
# teams = Team('',whole_season)

# 仅复赛后数据
teams = Team('',later_season)


teams_df = pd.concat([teams.mov, teams.tm_pace, teams.tm_ortg, teams.tm_drtg, teams.tm_nrtg], axis=1)
teams_df.columns=['场均净胜分MOV', 'Pace', 'OffensiveRating', 'DefensiveRating', 'NetRating']
teams_df = teams_df.sort_values(by='NetRating',ascending=False)
teams_df.round(1)

,场均净胜分MOV,Pace,OffensiveRating,DefensiveRating,NetRating
球队,,,,,
广东,26.4,100.2,124.4,97.3,27.0
辽宁,15.8,95.0,121.3,104.3,17.0
新疆,14.7,93.4,111.1,95.0,16.1
北京,11.0,88.1,108.3,95.6,12.8
浙江,9.2,92.4,114.4,104.2,10.2
广厦,8.4,96.0,111.9,102.8,9.1
福建,2.3,99.6,114.6,112.3,2.4
北控,-2.5,94.2,107.7,110.4,-2.7
上海,-2.6,92.3,106.3,109.2,-2.9


# Model Coeficients 

Reference:
* https://medium.com/hackernoon/how-to-predict-the-nba-with-a-machine-learning-system-written-in-python-part-ii-f276b19520b9
* https://www.basketball-reference.com/about/factors.html
* https://www.basketball-reference.com/about/glossary.html

In [145]:
coefs = dict({})

# home court advantage: 0.10218887
# 因为是隔离比赛，所以没有主场优势
coefs['home_court_advantage'] = 0

# effective_field_goal_percentage_difference
coefs['effective_field_goal_percentage_difference']= 0.16118265
tm_3pg = teams.all_games_stats.groupby('球队')['3分中'].sum()
effective_field_goal_percentage = 100*(teams.tm_fg + 0.5 * tm_3pg) / teams.tm_fga

# coef_turnover_percentage_difference
coefs['turnover_percentage_difference']=-0.05958713
tm_tov = teams.all_games_stats.groupby('球队')['失误'].sum()
# below definition is from BBR and 4 factors by Dean Oliver
turnover_percentage = 100*tm_tov / (teams.tm_fga + 0.44 * teams.tm_fta + tm_tov)

# coef_offensive_rebound_percentage_difference
coefs['offensive_rebound_percentage_difference']=0.07061777
# below definition is from BBR
# offensive_rebound_percentage = 100 * (teams.tm_orb * (teams.tm_mp / 5)) / (teams.tm_mp*2 * (teams.tm_orb + teams.op_tm_orb))

# below definition is from 4 factors by Dean Oliver
offensive_rebound_percentage = 100*(teams.tm_orb/(teams.tm_orb+teams.op_tm_drb))

coefs['free_throws_to_field_goals_attempts_difference']=0.03267933
tm_ftg = teams.all_games_stats.groupby('球队')['罚球中'].sum()

# author didn't say but in 4 factors it's a percentage, 
# we'll decide later when result is out
free_throws_to_field_goals_attempts = 100*tm_ftg/teams.tm_fga

# no one is traveling
# coefs['distance_traveled_in_last_7_days_difference']=-0.01459163
coefs['distance_traveled_in_last_7_days_difference']=0

# looks like it's win lose last seven matches per https://www.transfermarkt.us/premier-league/formtabelle/wettbewerb/GB1
# coefs['form_in_last_seven_matches_difference']=0.0828436
coefs['form_in_last_seven_matches_difference']=0

coefs['offensive_rating_difference']=0.17885523
offensive_rating = teams.tm_ortg

coefs['defensive_rating_difference']=-0.33924331
defensive_rating = teams.tm_drtg

# no court advantage at this moment

# coefs['effective_field_goal_percentage_difference_court']=0.10808104
# coefs['turnover_percentage_difference_court']=-0.09548481
# coefs['offensive_rebound_percentage_difference_court']=0.07055131
# coefs['free_throws_to_field_goals_attempts_difference_court']=0.0748545
# coefs['form_in_last_seven_matches_difference_court']=-0.00486437
# coefs['offensive_rating_difference_court']=0.14822224
# coefs['defensive_rating_difference_court']=-0.21756487

import math

def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [146]:
schedules = [('青岛','山东'),
('广厦','福建'),
('北控','吉林'),
('浙江','山西')]

for schedule in schedules:
    home_team = schedule[0]
    away_team = schedule[1]
    print (home_team+' V.S '+away_team)

    Xs = coefs.copy()
    Xs['home_court_advantage'] = 1
    Xs['effective_field_goal_percentage_difference'] = effective_field_goal_percentage[home_team]-effective_field_goal_percentage[away_team]
    Xs['turnover_percentage_difference']= turnover_percentage[home_team] - turnover_percentage[away_team]
    Xs['offensive_rebound_percentage_difference']= offensive_rebound_percentage[home_team]-offensive_rebound_percentage[away_team]
    Xs['free_throws_to_field_goals_attempts_difference'] = free_throws_to_field_goals_attempts[home_team]-free_throws_to_field_goals_attempts[away_team]
    Xs['distance_traveled_in_last_7_days_difference']= 0
    Xs['form_in_last_seven_matches_difference']= 0
    Xs['offensive_rating_difference'] = offensive_rating[home_team] - offensive_rating[away_team]
    Xs['defensive_rating_difference'] = defensive_rating[home_team] - defensive_rating[away_team]
    
    Z = 0
    for key in coefs:
        Z+= coefs[key]*Xs[key]
        
    print(sigmoid(Z))

青岛 V.S 山东
0.29892010501084254
广厦 V.S 福建
0.9394443523572833
北控 V.S 吉林
0.7907049616637225
浙江 V.S 山西
0.9945878509482465


# 用整个赛季数据预测：

    青岛 V.S 山东
    0.41543370748160735
    广厦 V.S 福建
    0.9125730862476105
    北控 V.S 吉林
    0.6327203272535492
    浙江 V.S 山西
    0.8515366750830545

# 用复赛后数据预测：

    青岛 V.S 山东
    0.29892010501084254
    广厦 V.S 福建
    0.9394443523572833
    北控 V.S 吉林
    0.7907049616637225
    浙江 V.S 山西
    0.9945878509482465